In [225]:
import os
import sys
import importlib

import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import redfin

importlib.reload(redfin)

pd.set_option('display.max_columns', None)

In [39]:
!pwd
!ls


/Users/timcodes/ds/metis/metisgh/02_Luther/Luther
2013_movies.csv                geckodriver.log
LICENSE                        pickles
README.md                      rappold_tim_challenge_3.ipynb
__pycache__                    redfin.py
data.txt                       redfin_2018-04-18-08-51-54.csv
explore_luther.ipynb


['.DS_Store',
 'listing_urls_10024page_18.pkl',
 'listing_urls_11211page_18.pkl',
 'listing_urls_48503page_18.pkl',
 'listing_urls_77373page_18.pkl',
 'listing_urls_90049page_18.pkl',
 'listing_urls_90292page_18.pkl',
 'listing_urls_90301page_18.pkl',
 'listing_urls_90403page_18.pkl',
 'listing_urls_94110page_18.pkl',
 'listing_urls_94549page_18.pkl',
 'listing_urls_94605page_18.pkl',
 'listing_urls_94610page_18.pkl',
 'listing_urls_94611page_18.pkl',
 'listing_urls_95476page_18.pkl']

In [181]:
list_of_all_urls = redfin.load_everything_pickle()

In [182]:
len(list_of_all_urls)

4893

In [203]:
np.random.choice(small_list, size=7, replace=False, p=None)

array(['/CA/Sonoma/431-Chase-St-95476/home/2259714',
       '/CA/Lafayette/1008-Woodbury-Rd-94549/unit-101/home/112888202',
       '/NY/New-York/186-W-80th-St-10024/unit-7K/home/45273346',
       '/TX/Spring/5015-Fitzwater-Dr-77373/home/30314611',
       '/CA/Sonoma/790-Peru-Rd-95476/home/2497327',
       '/CA/Lafayette/3164-Cordova-Way-94549/home/997787',
       '/NY/New-York/331-W-87th-St-10024/home/45275791'], dtype='<U68')

In [205]:
pickle_file='pickles/home_stats_all.pkl'
with open(pickle_file, 'rb') as picklefile:
    dummydata = pickle.load(picklefile)

In [212]:
for i, dummy in enumerate(dummydata):
    if 

169

In [243]:
list(range(0,len(dummydata),25))


[0, 25, 50, 75, 100, 125, 150]

In [213]:
dummydata

[{'APN': '022180008',
  'Baths': '—',
  'Beds': '—',
  'County': 'Kings County',
  'Finished Sq. Ft.': '3,360',
  'Last Sold': 'Aug 2, 2017',
  'Lot Size': '1,627 Sq. Ft.',
  'Sales Price': '$1,550,000',
  'Stories': '3',
  'Style': 'Multi-Family (2-4 Unit)',
  'Total Sq. Ft.': '3,360',
  'Unfinished Sq. Ft.': '—',
  'Year Built': '1899',
  'Year Renovated': '1899',
  'Zip Code': '11211',
  'rURL': '/NY/Unknown/645-Bedford-Ave-11211/home/40736142'},
 {'APN': '0983910000234',
  'Baths': '2.5',
  'Beds': '3',
  'County': 'Harris County',
  'Finished Sq. Ft.': '2,844',
  'Last Sold': None,
  'Lot Size': '0.43 Acres',
  'Sales Price': None,
  'Stories': '2',
  'Style': 'Single Family Residential',
  'Total Sq. Ft.': '2,844',
  'Unfinished Sq. Ft.': '—',
  'Year Built': '1988',
  'Year Renovated': '1988',
  'Zip Code': '77373',
  'rURL': '/TX/Spring/650-Ames-St-77373/home/30205884'},
 {'APN': '4024031032',
  'Baths': '1',
  'Beds': '2',
  'County': 'Los Angeles County',
  'Finished Sq. Ft.'

In [222]:
test_list = redfin.load_everything_pickle()
test_list = np.random.choice(test_list, size=35, replace=False)

In [223]:
redfin.scrape_home_stats(test_list)

Processing link #0: /CA/Inglewood/507-W-Kelso-St-90301/home/6445595
Processing link #1: /CA/Marina-del-Rey/13650-Marina-Pointe-Dr-90292/unit-1804/home/17235582
Processing link #2: /CA/Oakland/7081-Elverton-Dr-94611/home/1342622
Processing link #3: /CA/Los-Angeles/200-N-Bristol-Ave-90049/home/6838373
Processing link #4: /NY/Unknown/179-Woodpoint-Rd-11211/unit-2A/home/113720143
Processing link #5: /CA/Lafayette/3-Mountain-View-Pl-94549/home/1001479
Processing link #6: /CA/Lafayette/3363-Moraga-Blvd-94549/home/571536
Processing link #7: /CA/Oakland/6425-Melville-Dr-94611/home/1703379
Processing link #8: /CA/San-Francisco/3280-17th-St-94110/home/109042612
Processing link #9: /CA/Marina-del-Rey/4316-Marina-City-Dr-90292/unit-823/home/18580595
Processing link #10: /TX/Spring/23438-Stargazer-Pt-77373/home/52556977
Processing link #11: /CA/San-Francisco/176-Liberty-St-94110/home/1192426
Processing link #12: /CA/Marina-del-Rey/679-Washington-Blvd-90292/home/6735148
Processing link #13: /CA/Los-

[{'APN': '4018018019',
  'Baths': '1',
  'Beds': '3',
  'County': 'Los Angeles County',
  'Finished Sq. Ft.': '1,186',
  'Last Sold': 'Sep 25, 2015',
  'Lot Size': '7,496 Sq. Ft.',
  'Sales Price': '$380,000',
  'Stories': '1',
  'Style': 'Single Family Residential',
  'Total Sq. Ft.': '1,186',
  'Unfinished Sq. Ft.': '—',
  'Year Built': '1940',
  'Year Renovated': '1940',
  'Zip Code': '90301',
  'rURL': '/CA/Inglewood/507-W-Kelso-St-90301/home/6445595'},
 {'APN': '4229021130',
  'Baths': '4',
  'Beds': '3',
  'County': 'Los Angeles County',
  'Finished Sq. Ft.': '3,245',
  'Last Sold': 'Jan 30, 2017',
  'Lot Size': '—',
  'Sales Price': '$3,850,000',
  'Stories': '—',
  'Style': 'Condo/Co-op',
  'Total Sq. Ft.': '3,245',
  'Unfinished Sq. Ft.': '—',
  'Year Built': '2006',
  'Year Renovated': '2006',
  'Zip Code': '90292',
  'rURL': '/CA/Marina-del-Rey/13650-Marina-Pointe-Dr-90292/unit-1804/home/17235582'},
 {'APN': '048G744801603',
  'Accessible': 'No',
  'Basement': 'No',
  'Baths

In [228]:
ahs = redfin.load_all_home_stats()

In [229]:
len(ahs)

30

In [221]:
pd.DataFrame(all_stats)

,APN,Baths,Beds,County,Finished Sq. Ft.,Last Sold,Lot Size,Sales Price,Stories,Style,Total Sq. Ft.,Unfinished Sq. Ft.,Year Built,Year Renovated,Zip Code,rURL
0,4117177004,1.5,3,Genesee County,"1,830","Dec 28, 2016","9,148 Sq. Ft.","$113,700",2,Single Family Residential,"2,215",385,1924,—,48503,/MI/Flint/1619-Woodlawn-Park-Dr-48503/home/102...
1,043A464802600,2,4,Alameda County,"2,354","Jul 1, 2017","6,962 Sq. Ft.","$760,000",1,Single Family Residential,"2,354",—,1938,1967,94605,/CA/Oakland/8301-Aster-Ave-94605/home/1417868
2,048685700912,2,3,Alameda County,"2,099","Dec 14, 2016",0.38 Acres,"$880,000",1,Single Family Residential,"2,099",—,1954,1954,94605,/CA/Oakland/4135-Sequoyah-Rd-94605/home/1533512
3,4230005122,2,—,Los Angeles County,"1,540","Sep 8, 2015",—,"$905,000",—,Condo/Co-op,"1,540",—,2008,2008,90292,/CA/Marina-del-Rey/4141-Glencoe-Ave-90292/unit...
4,5739010,1,—,San Francisco County,973,"Jul 17, 2015","1,751 Sq. Ft.","$995,000",1,Single Family Residential,973,—,1950,—,94110,/CA/San-Francisco/4156-Folsom-St-94110/home/17...
5,4117176005,2,3,Genesee County,"2,523","Dec 27, 2017","9,583 Sq. Ft.","$129,900",2,Single Family Residential,"3,063",540,1935,—,48503,/MI/Flint/1514-Linwood-Ave-48503/home/102570075
6,1255380040028,2.5,3,Harris County,"1,564",None,"3,025 Sq. Ft.",None,2,Townhouse,"1,564",—,2005,2005,77373,/TX/Spring/23731-Pebworth-Pl-77373/home/29327557
7,0983910000234,2.5,3,Harris County,"2,844",None,0.43 Acres,None,2,Single Family Residential,"2,844",—,1988,1988,77373,/TX/Spring/650-Ames-St-77373/home/30205884
8,4281025048,2,2,Los Angeles County,"1,470","Feb 28, 2018",—,"$1,273,000",—,Condo/Co-op,"1,470",—,1970,1970,90403,/CA/Santa-Monica/901-10th-St-90403/unit-201/ho...
9,051471302500,3,3,Alameda County,"2,848","Jul 9, 2015",0.26 Acres,"$1,876,000",1,Single Family Residential,"2,848",—,1955,1968,94611-3541,/CA/Piedmont/18-Littlewood-Dr-94611/home/620471


In [197]:
d = [{'points': 50, 'time': '5:00', 'year': 2010}, 
{'points': 25, 'time': '6:00', 'month': "february"}, 
{'points':90, 'time': '9:00', 'month': 'january'}, 
{'points_h1':20, 'month': 'june'}]

In [198]:
pd.DataFrame(d)

,month,points,points_h1,time,year
0,NaN,50.0,NaN,5:00,2010.0
1,february,25.0,NaN,6:00,NaN
2,january,90.0,NaN,9:00,NaN
3,june,NaN,20.0,NaN,NaN


In [173]:
columns = ['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY', 'STATE', 'ZIP',
           'PRICE', 'BEDS', 'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT',
           'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
           'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED',
           'LATITUDE', 'LONGITUDE']

In [174]:
pd.Series(index=columns)

SALE TYPE                    NaN
SOLD DATE                    NaN
PROPERTY TYPE                NaN
ADDRESS                      NaN
CITY                         NaN
STATE                        NaN
ZIP                          NaN
PRICE                        NaN
BEDS                         NaN
BATHS                        NaN
LOCATION                     NaN
SQUARE FEET                  NaN
LOT SIZE                     NaN
YEAR BUILT                   NaN
DAYS ON MARKET               NaN
$/SQUARE FEET                NaN
HOA/MONTH                    NaN
STATUS                       NaN
NEXT OPEN HOUSE START TIME   NaN
NEXT OPEN HOUSE END TIME     NaN
URL                          NaN
SOURCE                       NaN
MLS#                         NaN
FAVORITE                     NaN
INTERESTED                   NaN
LATITUDE                     NaN
LONGITUDE                    NaN
dtype: float64

In [ ]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

## Figure out Filtered Landing Page
- Verify the problem using requests and bs4: get home_listing_url and verify that its status is `active` or `new` and not `sold`
- Recreate get request using chrome driver and selenium:
Assuming you are on the page you want to parse, Selenium stores the source HTML in the driver's page_source attribute. You would then load the page_source into BeautifulSoup as follows:

```
In [8]: from bs4 import BeautifulSoup

In [9]: from selenium import webdriver

In [10]: driver = webdriver.Firefox()

In [11]: driver.get('http://news.ycombinator.com')

In [12]: html = driver.page_source

In [13]: soup = BeautifulSoup(html)

In [14]: for tag in soup.find_all('title'):
   ....:     print tag.text
   ....:  
```


In [232]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
import sys

chromedriver = "~/Downloads/chromedriver" # path to the chromedriver executable
chromedriver = os.path.expanduser(chromedriver)
print('chromedriver path: {}'.format(chromedriver))
sys.path.append(chromedriver)

driver = webdriver.Chrome(chromedriver)


chromedriver path: /Users/timcodes/Downloads/chromedriver


In [235]:
chrome_url = construct_redfin_url('94605', filter_='sold-3yr')
chrome_url

'https://www.redfin.com/zipcode/94605/filter/include=sold-3yr'

In [239]:
driver.get(chrome_url)
html = driver.page_source
selenium_soup = BeautifulSoup(html,"lxml")

selenium_list = find_home_listing_urls(selenium_soup)

In [240]:
selenium_list

['/CA/Oakland/2812-76th-Ave-94605/home/2036652',
 '/CA/Oakland/6129-Old-Quarry-Loop-94605/home/12115918',
 '/CA/Oakland/5921-Hilton-St-94605/home/568204',
 '/CA/Oakland/3055-Bantry-Ave-94605/home/1161573',
 '/CA/Oakland/4009-Malcolm-Ave-94605/home/1993133',
 '/CA/Oakland/8301-Aster-Ave-94605/home/1417868',
 '/CA/Oakland/8009-Earl-St-94605/home/579005',
 '/CA/Oakland/5528-Picardy-Dr-S-94605/home/567257',
 '/CA/Oakland/5514-Picardy-Dr-N-94605/home/1563765',
 '/CA/Oakland/7841-Surrey-Ln-94605/home/592451',
 '/CA/Oakland/4751-Dunkirk-Ave-94605/home/661538',
 '/CA/Oakland/3101-Partridge-Ave-94605/home/2040553',
 '/CA/Oakland/6144-Old-Quarry-Loop-94605/home/21670339',
 '/CA/Oakland/7560-Hansom-Dr-94605/home/843353',
 '/CA/Oakland/3240-62nd-Ave-94605/home/1925170',
 '/CA/Oakland/3680-Malcolm-Ave-94605/home/1930765',
 '/CA/Oakland/3200-Revere-Ave-94605/home/599097',
 '/CA/Oakland/60-Maggiora-Dr-94605/home/981030',
 '/CA/Oakland/2-Riviera-Ct-94605/home/1952844',
 '/CA/Oakland/5115-Fairhill-Ct-9

### Open downloaded CSV from Redfin to check its contents and use as a reference

In [37]:
!ls

LICENSE                        explore_luther.ipynb
README.md                      redfin_2018-04-18-08-51-54.csv


In [119]:
rf_csv = pd.read_csv('redfin_2018-04-18-08-51-54.csv')
rf_csv.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY', 'STATE',
       'ZIP', 'PRICE', 'BEDS', 'BATHS', 'LOCATION', 'SQUARE FEET', 'LOT SIZE',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'HOA/MONTH', 'STATUS',
       'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME',
       'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [174]:
rf_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 27 columns):
SALE TYPE                                                                                     350 non-null object
SOLD DATE                                                                                     294 non-null object
PROPERTY TYPE                                                                                 350 non-null object
ADDRESS                                                                                       343 non-null object
CITY                                                                                          350 non-null object
STATE                                                                                         350 non-null object
ZIP                                                                                           350 non-null object
PRICE                                                                                         350

In [178]:
rf_csv.head()
list(rf_csv.columns)

['SALE TYPE',
 'SOLD DATE',
 'PROPERTY TYPE',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'PRICE',
 'BEDS',
 'BATHS',
 'LOCATION',
 'SQUARE FEET',
 'LOT SIZE',
 'YEAR BUILT',
 'DAYS ON MARKET',
 '$/SQUARE FEET',
 'HOA/MONTH',
 'STATUS',
 'NEXT OPEN HOUSE START TIME',
 'NEXT OPEN HOUSE END TIME',
 'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
 'SOURCE',
 'MLS#',
 'FAVORITE',
 'INTERESTED',
 'LATITUDE',
 'LONGITUDE']

In [184]:
rf_csv.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE,ZIP,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,April-4-2018,Single Family Residential,5514 Picardy Dr,Oakland,CA,94605,800000,4.0,2.0,Normandy Gardens,1650.0,3230.0,1926.0,14.0,485.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/5514-Picardy-...,EBRD/CCAR/Bay East,40815087,N,Y,37.775837,-122.190547
1,PAST SALE,April-16-2018,Multi-Family (2-4 Unit),2812 76th Ave,Oakland,CA,94605,555000,4.0,3.0,Eastmont Area,2576.0,4068.0,1940.0,2.0,215.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/2812-76th-Ave...,EBRD/CCAR/Bay East,40808499,N,Y,37.767668,-122.169261
2,PAST SALE,September-5-2017,Single Family Residential,4009 Malcolm Ave,Oakland,CA,94605,740000,3.0,2.5,Chabot Park,1948.0,9086.0,1960.0,225.0,380.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/4009-Malcolm-...,EBRD/CCAR/Bay East,40787391,N,Y,37.750808,-122.134996
3,PAST SALE,July-1-2017,Single Family Residential,8301 Aster Ave,Oakland,CA,94605,760000,4.0,2.0,Oakland,2354.0,6962.0,1938.0,291.0,323.0,NaN,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/8301-Aster-Av...,EBRD/CCAR/Bay East,40781777,N,Y,37.764025,-122.160524
4,PAST SALE,May-10-2017,Condo/Co-op,6144 Old Quarry Loop,Oakland,CA,94605,505000,2.0,2.0,East Oakl Hills,1079.0,NaN,2007.0,343.0,468.0,282.0,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/6144-Old-Quar...,EBRD/CCAR/Bay East,40775089,N,Y,37.778864,-122.165009
